In [42]:
!pip install "modin[ray]"


In [43]:
pip install -U ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [46]:
!pip install "dask[dataframe]"


In [104]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
from ray import init
import yaml
import os
import time

In [106]:
start_time = time.time()
p_df = pd.read_csv('HAM10000_metadata.csv')
end_time = time.time()
print(f"Pandas: {end_time - start_time} seconds")

Pandas: 0.060296058654785156 seconds


In [108]:
start_time = time.time()
d_df = dd.read_csv('HAM10000_metadata.csv')
end_time = time.time()

print(f"Dask: {end_time - start_time} seconds")

Dask: 0.0008199214935302734 seconds


Here Dask is performing better pandas.

In [122]:
def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9]+', '_').str.strip().str.lower()
    return df

# Clean column names for each dataframe
p_df = clean_column_names(p_df)
d_df.columns = clean_column_names(d_df).columns

In [128]:
schema = {
    'separator': ',',
    'columns': list(d_df.columns)
}

with open('schema.yaml', 'w') as file:
    yaml.dump(schema, file)


In [132]:
with open('schema.yaml', 'r') as file:
    loaded_schema = yaml.safe_load(file)

assert len(d_df.columns) == len(loaded_schema['columns']), "Column count mismatch"
assert all([a == b for a, b in zip(d_df.columns, loaded_schema['columns'])]), "Column names mismatch"


In [136]:
import gzip

# Print the schema to understand its structure
print(schema)

# Adjust according to the actual structure
# Assuming you need to access a key directly or handle missing keys
output_separator = schema.get('output_separator', '|')  # Use a default separator if the key is not found
output_file_path = "processed_skin_cancer_dataset.txt.gz"

# Write the processed file in pipe-separated format and compress with gzip
with gzip.open(output_file_path, 'wt') as f:
    df.to_csv(f, sep=output_separator, index=False)


{'separator': ',', 'columns': ['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization']}


In [140]:
import os

# Create a summary
summary = {
    'total_rows': len(d_df),
    'total_columns': len(d_df.columns),
    'file_size': os.path.getsize(output_file_path)
}

# Print the summary
print("Summary of the processed file:")
print(f"Total Rows: {summary['total_rows']}")
print(f"Total Columns: {summary['total_columns']}")
print(f"File Size: {summary['file_size']} bytes")


Summary of the processed file:
Total Rows: 10015
Total Columns: 7
File Size: 77387 bytes
